In [ ]:
# Make sure to have the latest version of packages. In order to use latest environment, 
# choose "Always use latest environment" option in Notebook options

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq duckduckgo_search
    !pip install -Uqq fastai
    
# make sure you have python 3.10 and duckduckgo 3.9.5
import sys
print(sys.version)

import duckduckgo_search
print(duckduckgo_search.__version__)

In [ ]:
from itertools import islice
from duckduckgo_search import DDGS
from fastcore.all import *


def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    # DDGS().images return an interator containing all images found
    # we use islice to limit the number of results returned
    return L(islice(DDGS().images(term), max_images)).itemgot('image')

In [ ]:
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('giraffe photos', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'giraffe.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('oceanview photos', max_images=1)[0], 'oceanview.jpg', show_progress=False)
Image.open('oceanview.jpg').to_thumb(256,256)

In [ ]:
searches = 'oceanview','giraffe'
path = Path('giraffe_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=12)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

In [ ]:
is_giraffe,_,probs = learn.predict(PILImage.create('oceanview.jpg'))
print(f"This is a: {is_giraffe}.")
print(f"Probability it's a giraffe: {probs[0]:.4f}")